<a href="https://colab.research.google.com/github/kynemre/DataMining/blob/main/03_Sat%C4%B1%C5%9F_Stratejisi_i%C3%A7in_E_ticaret_Veri_Analizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kmart Ürün Satışları

<p>
<center>
<a href="https://en.wikipedia.org/wiki/Kmart"><img src="https://cdn.freebiesupply.com/logos/large/2x/kmart-1-logo-png-transparent.png" alt="Kmart Logo" title="Kmart Şirketi Hakkında Daha Fazla Bilgi" height=200></a><br>
<small>2004'ten günümüze Kmart Logosu</small>
</center>
</p><br>

<p>
Kmart Şirketi, Hoffman Estates, Illinois, ABD merkezli, marketler zinciridir. Şirket 1899'da S. S. Kresge Corporation adıyla kuruldu ve 1977'de bugünkü ismini aldı. Kmart birçok kez iflas etmiştir. 2019 ve 2020'deki hızlı mağaza kapanışları ve 2020 COVID-19 salgını sırasında yaşanan kapanışlar nedeniyle bugünkü mağaza sayısı belirsiz. <a href="https://en.wikipedia.org/wiki/Kmart" title="Kaynak"><small><sup>Vikipedi</sup></small></a> <br>İnternet Mağazacılığı: <a href="https://www.kmart.com"/>kmart.com</a>
</p><hr><br>
<p>
Bu çalışmadaki hedef, gelecek dönemlerde satış stratejisi geliştirilmesine yardımcı olmak için yapılan satışların görselleştirilmesidir. 
</p>
<p>
Kullanacağımız veri seti, 2019 yılında gerçekleşmiş olan bazı ürün satışları hakkında veriler içermektedir. Bu veriler şu başlıklar altında toplanmıştır;
<ul>
<li>Sipariş Numarası</li>
<li>Ürün İsmi</li>
<li>Sipariş Adedi</li>
<li>Ürün Fiyatı</li>
<li>Sipariş Tarihi</li>
<li>Sipariş Adresi</li>
</ul>
</p>
<p>
<small>
Veri seti: <a href="https://github.com/kynemre/DataMining/blob/main/data/Sales_Data.zip" title="zip dosyasına git">GitHub - kynemre</a>
</small>
</p>

#Veri Setinin İçe Aktarılması

<h3>Veri Setini İndirilmesi ve ZIP Dosyasını Açılması</h3>

In [ ]:
adress = 'https://raw.githubusercontent.com/kynemre/DataMining/main/data/Sales_Data.zip';
!wget -q -O veri.zip "{adress}";#Verileri indirdik
!unzip -q veri.zip
!rm veri.zip ##Zip dosyasini sildik

<h3>Kütüphanelerin İçe Aktarılması<h3>

In [ ]:
import pandas as pd #Dosya okuma ve veri işlemleri 
import glob #Dosya yollarını/ isimlerini alma 
from tqdm import tqdm #İlerleme çubuğu
import datetime #Tarih işlemleri
from geopy.geocoders import Nominatim #Şehir koodinatlarını bulma
import plotly.express as px #Veri görselleştirme

<h3>Veri Seti Dosyalarının Okunması</h3>

In [ ]:
path = r'/content/Sales_Data' #Veri dosyalarının bulunduğu klasör
all_files = glob.glob(path + "/*.csv") #Dosya isimlerini tutan liste

li = [] #Okunan verileri tutan liste 

for filename in tqdm(all_files): #Veri dosyalarının okunması
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

data_org = pd.concat(li, axis=0, ignore_index=True) #Verilerin birleştirilmesi
del all_files, df, li #Gereksiz değişkenlerin silinmesi

100%|██████████| 12/12 [00:00<00:00, 29.80it/s]


In [ ]:
data = data_org.copy() #Değişikler için veri setinin kopyalanması
data.head(7) #Veri önizlemesi

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"
5,141239,AAA Batteries (4-pack),1,2.99,01/29/19 20:22,"775 Willow St, San Francisco, CA 94016"
6,141240,27in 4K Gaming Monitor,1,389.99,01/26/19 12:16,"979 Park St, Los Angeles, CA 90001"


#Veri Setinin Temizlenmesi

<h3>Boş Değerlerin Temizlenmesi</h3>

In [ ]:
data.isna().value_counts() #Eksik örnek adetinin bulunması

Order ID  Product  Quantity Ordered  Price Each  Order Date  Purchase Address
False     False    False             False       False       False               186305
True      True     True              True        True        True                   545
dtype: int64

<h4>186,305 adet örnekten 545 adet örnek eksik veri içeriyor.</h4>

In [ ]:
data.dropna(inplace=True) #Eksik/boş örneklerin silinmesi
data.isna().any()

Order ID            False
Product             False
Quantity Ordered    False
Price Each          False
Order Date          False
Purchase Address    False
dtype: bool

<h3>Hatalı Örneklerin Silinmesi ve Verilerin Tarihe Göre Sıralanması</h3>

In [ ]:
data[data["Order Date"] == "Order Date"].head() #Hatalı örneklerin önizlemsi

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1073,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1102,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1194,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1897,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2463,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [ ]:
misinfo_count = len(data) - len(data[data["Order Date"] != "Order Date"]) #Hatalı örneklerin sayılması
#Hatalı örneklerin silinmesi ve veri setinin tarihe göre sıralanması
data = data[data["Order Date"] != "Order Date"].sort_values("Order Date").sort_values("Order Date")
print("""{} adet hatalı örnek girilmiş
Hatalı satırlar silindi.
Veriler tarihe göre sıralandı.""".format(misinfo_count))

355 adet hatalı örnek girilmiş
Hatalı satırlar silindi.
Veriler tarihe göre sıralandı.


<h3>İndekslerin ve Veri Tiplerinin Düzenlenmesi</h3>

In [ ]:
data.reset_index(drop=True, inplace=True) #İndeks değerlerini sıfırlama 
data[['Order ID','Quantity Ordered']] = data[['Order ID',
                             'Quantity Ordered']].astype(int) #Veri tiplerinin düzenlenmesi
data['Price Each'] = data['Price Each'].astype(float) #Veri tiplerinin düzenlenmesi
data.tail() #Veri önizleme

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
185945,298406,AAA Batteries (4-pack),2,2.99,12/31/19 23:42,"30 Elm St, San Francisco, CA 94016"
185946,297481,AA Batteries (4-pack),1,3.84,12/31/19 23:42,"82 Hill St, Dallas, TX 75001"
185947,317048,AAA Batteries (4-pack),2,2.99,12/31/19 23:45,"743 Adams St, San Francisco, CA 94016"
185948,309773,AAA Batteries (4-pack),1,2.99,12/31/19 23:52,"60 Hickory St, Los Angeles, CA 90001"
185949,297674,AAA Batteries (4-pack),1,2.99,12/31/19 23:53,"425 Lake St, Portland, OR 97035"


#Veri Görselleştirme - Aylara Göre Satış

In [ ]:
monthly_income = dict() #Aylık geliri tutacak obje
for i in tqdm(range(1,13)): #Her aydaki toplam geliri hesaplayan döngü
    #Bir aylık verinin tutulması
    df_monthly = data[data["Order Date"].str.split("/",expand=True)[0].astype(int) == i]
    month = datetime.date(1900, i, 1).strftime('%B') #Ay ismini alma ör: 4->April
    #Aylık verinin toplanması ve objeye eklenmesi
    monthly_income[month] = (df_monthly["Quantity Ordered"]*df_monthly["Price Each"]).sum().round(2)
    del df_monthly
monthly_income = pd.DataFrame(monthly_income.items(),
             columns=['Month', 'Income']) #Aylık gelir verilerinin df'e aktarılması
monthly_income #Önizleme

100%|██████████| 12/12 [00:08<00:00,  1.48it/s]


,Month,Income
0,January,1822256.73
1,February,2202022.42
2,March,2807100.38
3,April,3390670.24
4,May,3152606.75
5,June,2577802.26
6,July,2647775.76
7,August,2244467.88
8,September,2097560.13
9,October,3736726.88


In [ ]:
fig = px.bar(monthly_income,
             x='Month',
             y='Income',
             color_discrete_sequence=['#209910'],
             title="Aylık Gelir Sütun Grafiği") #Sütun grafiğinin çizilmesi
fig.show()

#Veri Görselleştirme - Konuma Göre Satış

In [ ]:
#Koordinat bilgilerini almak için Nominatim objesi oluşturulması
geolocator = Nominatim(user_agent="http") 
lat=[] #Enlemler listesi
lon=[] #Boylamlar listesi
city_income = dict() #Şehir gelirini tutacak olan objenin oluşturulması
#Şehir isimlerinin tutan liste
city_list = data["Purchase Address"].str.split(',',expand=True)[1].value_counts().keys()
for name in tqdm(city_list): #Her şehirdeki toplam geliri hesaplayan döngü
    #Bir şehire yapılan satışların bulunması
    df_city = data[data["Purchase Address"].str.split(',',expand=True)[1] == name]
    #Yapılan satışların toplanması ve objeye eklenmesi
    city_income[name] = (df_city["Quantity Ordered"]*df_city["Price Each"]).sum().round(2)
    lat.append(geolocator.geocode(name)[1][0]) #Koordinat bilgilerinin listeye eklenmesi
    lon.append(geolocator.geocode(name)[1][1]) #Koordinat bilgilerinin listeye eklenmesi
    del df_city
city_income = pd.DataFrame(city_income.items(),
             columns=['City', 'Income']) #Şehir gelir verilerinin df'e aktarılması
city_income["lat"] = lat #Şehir enlem verilerinin df'e aktarılması
city_income["lon"] = lon #Şehir boylam verilerinin df'e aktarılması
del lat, lon, geolocator
city_income #Önizleme

100%|██████████| 9/9 [00:22<00:00,  2.47s/it]


,City,Income,lat,lon
0,San Francisco,8262203.91,37.779026,-122.419906
1,Los Angeles,5452570.80,34.053691,-118.242766
2,New York City,4664317.43,40.712728,-74.006015
3,Boston,3661642.01,42.360253,-71.058291
4,Atlanta,2795498.58,33.748992,-84.390264
5,Dallas,2767975.40,32.776272,-96.796856
6,Seattle,2747755.48,47.603832,-122.330062
7,Portland,2320490.61,45.520247,-122.674195
8,Austin,1819581.75,30.271129,-97.743700


In [ ]:
fig = px.bar(city_income,
             x='City',
             y='Income',
             color_discrete_sequence=['#1b98e0'],
             title="Şehir Gelirleri Sütun Grafiği") #Sütun grafiğinin çizilmesi
fig.show()

In [ ]:
fig = px.scatter_mapbox(city_income, lat='lat', lon='lon', 
                        hover_name="City",
                        size="Income",
                        size_max=35, 
                        zoom=3,
                        opacity=.85,
                        color_discrete_sequence=['#9d0208'],
                        mapbox_style='open-street-map',
                        title='Şehirlere Göre Toplam Gelir') #Harita dağılım grafiğinin çizilmesi
fig.update_layout(showlegend=False) #Grafik ayarları - Lejantın kapatılması
fig.show()

#Veri Görselleştirme - Saate Göre Satış

In [ ]:
hourly_income = dict()#Saatlik geliri tutacak obje
for i in tqdm(range(0,24)):#Her saatteki toplam geliri hesaplayan döngü
    #Bir saatlik verinin tutulması
    df_hourly = data[data["Order Date"].str.split(" ",expand=True)[1].str.split(":",expand=True).astype(int)[0] == i]
    #Saatlik verinin toplanması ve objeye eklenmesi
    hourly_income[str(i)] = (df_hourly["Quantity Ordered"]*df_hourly["Price Each"]).sum().round(2)
    del df_hourly
hourly_income = pd.DataFrame(hourly_income.items(),
             columns=['Hour', 'Income'])#Saatlik gelir verilerinin df'e aktarılması
hourly_income

100%|██████████| 24/24 [00:27<00:00,  1.15s/it]


,Hour,Income
0,0,713721.27
1,1,460866.88
2,2,234851.44
3,3,145757.89
4,4,162661.01
5,5,230679.82
6,6,448113.00
7,7,744854.12
8,8,1192348.97
9,9,1639030.58


In [ ]:
fig = px.bar(hourly_income,
             x='Hour',
             y='Income',
             title="Saatlik Gelir Sütun Grafiği") #Sütun grafiğinin çizilmesi
fig.update_layout(showlegend=False)
fig.show()

#Veri Görselleştirme - Ürüne Göre Satış

In [ ]:
#Ürünlerin kaç adet satıldığının bulunması ve df'e aktarılması
product_sales = data.groupby('Product')["Quantity Ordered"].sum().reset_index()
product_sales.head() #Önizleme

,Product,Quantity Ordered
0,20in Monitor,4129
1,27in 4K Gaming Monitor,6244
2,27in FHD Monitor,7550
3,34in Ultrawide Monitor,6199
4,AA Batteries (4-pack),27635


In [ ]:
fig = px.bar(product_sales,
             x='Product',
             y='Quantity Ordered',
             title="Ürün Satış Adeti Sütun Grafiği",
             color="Product",
             color_discrete_sequence=px.colors.sequential.Magma) #Sütun Grafiğinin Çizilmesi
fig.update_layout(showlegend=False)
fig.show()